In [3]:
import ndjson
import os


file_path = r'C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\Export v2 project - Health_Care_Interaction - 10_30_2024.ndjson'


with open(file_path, 'r') as f:
    data = ndjson.load(f)


print("First annotation:", data[0])

# Process each annotation
validation_results = []
for annotation in data:
    result = HealthcareAnnotationValidator().validate_annotation(annotation)
    validation_results.append(result)

First annotation: {'data_row': {'id': 'cm2uh7qqp01860777dixjhfpx', 'external_id': 'healthcare_interactions.pdf', 'row_data': 'https://storage.labelbox.com/cm2lnf9vn0jic07x8f9a1ctdq%2F551fe425-9512-5857-025d-7a734ad693c8-healthcare_interactions.pdf?Expires=1730375767743&KeyName=labelbox-assets-key-3&Signature=cJBhpqTwn1rH9D9qt-R5d6e_5zE', 'details': {'dataset_id': 'cm2uh7okr001d076233cudoqc', 'dataset_name': 'Healthcare_interaction', 'created_at': '2024-10-29T13:21:16.131+00:00', 'updated_at': '2024-10-29T13:21:17.820+00:00', 'last_activity_at': '2024-10-30T11:51:19.846+00:00', 'created_by': 'mbavhamadoba@gmail.com'}}, 'media_attributes': {'asset_type': 'pdf', 'mime_type': 'application/pdf'}, 'attachments': [], 'metadata_fields': [], 'projects': {'cm2uh8pxt00ck07zd9epchhrd': {'name': 'Health_Care_Interaction', 'labels': [{'label_kind': 'Default', 'version': '1.0.0', 'id': 'cm2uh9zjk00gw07120pdpfqk8', 'label_details': {'created_at': '2024-10-30T11:51:19.000+00:00', 'updated_at': '2024-10

NameError: name 'HealthcareAnnotationValidator' is not defined

In [11]:
import json
import ndjson
from datetime import datetime
from typing import Dict, List
from collections import defaultdict
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import os

class NumpyJSONEncoder(json.JSONEncoder):
    """Custom JSON encoder for handling NumPy data types."""
    def default(self, obj):
        if isinstance(obj, np.generic):
            return obj.item()
        return super().default(obj)

class HealthcareAnnotationValidator:
    def __init__(self, schema_path: str = None):
        """Initialize validator with schema configuration."""
        if schema_path is None:
            schema_path = r'C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\labelbox_schema.json'
        
        try:
            with open(schema_path, 'r') as f:
                self.schema = json.load(f)
        except FileNotFoundError:
            raise FileNotFoundError(f"Schema file not found at: {schema_path}")
        except json.JSONDecodeError:
            raise ValueError(f"Invalid JSON in schema file at: {schema_path}")
        except Exception as e:
            raise Exception(f"Error loading schema file: {str(e)}")
        
        self._init_schema_lookups()
        
        self.kpi_targets = {
            'inter_annotator_agreement': 0.90,
            'medical_terminology_accuracy': 0.98,
            'service_classification_accuracy': 0.95,
            'provider_identification_accuracy': 0.98,
            'required_fields_completion': 1.0,
            'supporting_info_capture': 0.95,
            'patient_demographic_completion': 0.90,
            'geographic_coverage': 1.0,
            'annotations_per_hour': 40,
            'quality_verification_rate': 1.0,
            'error_correction_time': 24,
            'documentation_completeness': 1.0
        }

    def _init_schema_lookups(self):
        """Initialize lookup dictionaries for schema validation."""
        try:
            self.valid_entity_types = {
                option['value'] for tool in self.schema['tools'] 
                if tool['name'] == 'entities'
                for classification in tool['classifications']
                if classification['name'] == 'entity_type'
                for option in classification['options']
            }
            
            self.valid_relationship_types = {
                option['value'] for tool in self.schema['tools']
                if tool['name'] == 'relationships'
                for classification in tool['classifications']
                if classification['name'] == 'relationship_type'
                for option in classification['options']
            }
            
            self.valid_service_categories = {
                option['value'] for classification in self.schema['classifications']
                if classification['name'] == 'service_category'
                for option in classification['options']
            }
        except KeyError as e:
            raise ValueError(f"Invalid schema structure. Missing required key: {str(e)}")

    def validate_annotation(self, annotation: Dict) -> Dict:
        """
        Validate a single annotation against the schema.
        
        Args:
            annotation: Dictionary containing annotation data
            
        Returns:
            Dictionary containing validation results
        """
        validation_result = {
            'is_valid': True,
            'errors': [],
            'warnings': []
        }
        
        # Validate entities
        if 'entities' in annotation:
            for entity in annotation['entities']:
                # Check required fields
                required_fields = {'id', 'type', 'bbox', 'text'}
                missing_fields = required_fields - set(entity.keys())
                if missing_fields:
                    validation_result['is_valid'] = False
                    validation_result['errors'].append(
                        f"Missing required fields in entity: {missing_fields}")
                
                # Validate entity type
                if 'type' in entity and entity['type'] not in self.valid_entity_types:
                    validation_result['is_valid'] = False
                    validation_result['errors'].append(
                        f"Invalid entity type: {entity['type']}")
                
                # Validate bbox format
                if 'bbox' in entity:
                    if not (isinstance(entity['bbox'], list) and 
                            len(entity['bbox']) == 4 and 
                            all(isinstance(x, (int, float)) for x in entity['bbox'])):
                        validation_result['is_valid'] = False
                        validation_result['errors'].append(
                            f"Invalid bbox format for entity {entity['id']}")
        
        # Validate classifications
        if 'classifications' in annotation:
            if 'service_category' in annotation['classifications']:
                categories = annotation['classifications']['service_category']
                if not isinstance(categories, list):
                    categories = [categories]
                for category in categories:
                    if category not in self.valid_service_categories:
                        validation_result['is_valid'] = False
                        validation_result['errors'].append(
                            f"Invalid service category: {category}")
        
        return validation_result

    def _calculate_kpis(self, annotations: List[Dict], 
                       validation_results: List[Dict],
                       reference_annotations: List[Dict],
                       medical_terminology_dict: Dict,
                       start_time: float) -> Dict:
        """Calculate KPIs based on validation results and reference data."""
        kpi_results = {}
        
        # Calculate required fields completion rate
        valid_count = sum(1 for result in validation_results if result['is_valid'])
        kpi_results['required_fields_completion'] = valid_count / len(annotations)
        
        # Calculate inter-annotator agreement if reference annotations available
        if reference_annotations:
            agreement_scores = []
            for ann, ref in zip(annotations, reference_annotations):
                if 'entities' in ann and 'entities' in ref:
                    ann_labels = [e['type'] for e in ann['entities']]
                    ref_labels = [e['type'] for e in ref['entities']]
                    if ann_labels and ref_labels:
                        agreement = cohen_kappa_score(ann_labels, ref_labels)
                        agreement_scores.append(agreement)
            
            if agreement_scores:
                kpi_results['inter_annotator_agreement'] = np.mean(agreement_scores)
        
        # Calculate medical terminology accuracy if dictionary available
        if medical_terminology_dict:
            correct_terms = 0
            total_terms = 0
            for ann in annotations:
                if 'entities' in ann:
                    for entity in ann['entities']:
                        if entity['type'] in ['diagnosis', 'medication', 'procedure']:
                            total_terms += 1
                            if entity['text'].lower() in medical_terminology_dict:
                                correct_terms += 1
            
            if total_terms > 0:
                kpi_results['medical_terminology_accuracy'] = correct_terms / total_terms
        
        # Calculate processing speed
        processing_time = time.time() - start_time
        kpi_results['annotations_per_hour'] = (len(annotations) / processing_time) * 3600
        
        return kpi_results

    def _generate_summary_report(self, validation_results: List[Dict], 
                               kpi_results: Dict) -> Dict:
        """Generate a summary report of validation results and KPIs."""
        summary = {
            'total_annotations': len(validation_results),
            'valid_annotations': sum(1 for r in validation_results if r['is_valid']),
            'invalid_annotations': sum(1 for r in validation_results if not r['is_valid']),
            'kpi_metrics': kpi_results,
            'recommendations': []
        }
        
        # Generate recommendations based on KPI results
        for metric, value in kpi_results.items():
            if value < self.kpi_targets[metric]:
                summary['recommendations'].append(
                    f"Improve {metric}: Current {value:.2f} vs Target {self.kpi_targets[metric]}")
        
        return summary

    def generate_detailed_report(self, results: Dict) -> pd.DataFrame:
        """Generate a detailed report as a pandas DataFrame."""
        report_data = []
        
        for idx, (validation, kpis) in enumerate(zip(
            results['validation_results'], 
            [results['kpi_metrics']] * len(results['validation_results'])
        )):
            report_data.append({
                'annotation_id': idx + 1,
                'is_valid': validation['is_valid'],
                'error_count': len(validation['errors']),
                'warning_count': len(validation['warnings']),
                'errors': '; '.join(validation['errors']),
                'warnings': '; '.join(validation['warnings']),
                **kpis
            })
        
        return pd.DataFrame(report_data)

    def batch_process_annotations(self, annotations: List[Dict], 
                                reference_annotations: List[Dict] = None,
                                medical_terminology_dict: Dict = None) -> Dict:
        """Process multiple annotations in batch and calculate KPIs."""
        start_time = time.time()
        
        # Parallel processing of basic validation
        with ThreadPoolExecutor() as executor:
            validation_results = list(executor.map(self.validate_annotation, annotations))
        
        # Calculate KPIs
        kpi_results = self._calculate_kpis(
            annotations,
            validation_results,
            reference_annotations,
            medical_terminology_dict,
            start_time
        )
        
        # Generate summary
        summary = self._generate_summary_report(validation_results, kpi_results)
        
        # Convert NumPy types to Python native types
        summary = json.loads(json.dumps(summary, cls=NumpyJSONEncoder))
        kpi_results = json.loads(json.dumps(kpi_results, cls=NumpyJSONEncoder))
        
        return {
    'validation_results': validation_results,
    'kpi_metrics': kpi_results,
    'summary_report': summary
}

In [15]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, log_loss

def evaluate_model_performance(predicted_labels: List[str], true_labels: List[str]) -> Dict[str, float]:
    """
    Calculate and display key performance metrics of the model.
    
    Args:
        predicted_labels (List[str]): List of predicted labels.
        true_labels (List[str]): List of true labels.

    Returns:
        Dict[str, float]: Dictionary containing performance metrics.
    """
    performance_metrics = {}
    
    
    performance_metrics['accuracy'] = accuracy_score(true_labels, predicted_labels)
    
   
    performance_metrics['precision'] = precision_score(true_labels, predicted_labels, average='weighted')
    
    # Recall
    performance_metrics['recall'] = recall_score(true_labels, predicted_labels, average='weighted')
    
    # F1 Score
    performance_metrics['f1_score'] = f1_score(true_labels, predicted_labels, average='weighted')
    
    
    for metric, value in performance_metrics.items():
        print(f"{metric.capitalize()}: {value:.4f}")

    return performance_metrics


#model_performance = evaluate_model_performance(predicted_labels, true_labels)


In [19]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_model_performance(predicted_labels: List[str], true_labels: List[str], predicted_probabilities: List[List[float]] = None):
    """
    Calculate and display key performance metrics of the model.
    
    Args:
        predicted_labels (List[str]): List of predicted labels.
        true_labels (List[str]): List of true labels.
        predicted_probabilities (List[List[float]], optional): Probability predictions for log loss. Defaults to None.

    Returns:
        Dict[str, float]: Dictionary containing performance metrics.
    """
    performance_metrics = {}
    
    # Accuracy
    accuracy = accuracy_score(true_labels, predicted_labels)
    performance_metrics['accuracy'] = accuracy
    print(f"Accuracy: {accuracy:.4f}")
    
    # Precision
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    performance_metrics['precision'] = precision
    print(f"Precision: {precision:.4f}")
    
    # Recall
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    performance_metrics['recall'] = recall
    print(f"Recall: {recall:.4f}")
    
    # F1 Score
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    performance_metrics['f1_score'] = f1
    print(f"F1 Score: {f1:.4f}")
    
    # Log Loss (optional, requires probability predictions)
    if predicted_probabilities is not None:
        logloss = log_loss(true_labels, predicted_probabilities)
        performance_metrics['log_loss'] = logloss
        print(f"Log Loss: {logloss:.4f}")
    
    return performance_metrics


# model_performance = evaluate_model_performance(predicted_labels, true_labels, predicted_probabilities)



In [21]:
# EDA Section
def perform_eda(data: pd.DataFrame, label_column: str):
    """
    Perform Exploratory Data Analysis (EDA) on the dataset.

    Args:
        data (pd.DataFrame): Dataset containing annotations.
        label_column (str): Column name of the target label.
    """
    print("\n--- Exploratory Data Analysis ---")
    
    # Display basic dataset information
    print("\nDataset Info:")
    print(data.info())
    
    # Display basic statistics
    print("\nDescriptive Statistics:")
    print(data.describe(include='all'))
    
    # Class distribution
    print("\nClass Distribution:")
    class_counts = data[label_column].value_counts()
    print(class_counts)
    plt.figure(figsize=(10, 6))
    sns.countplot(data=data, x=label_column, palette="viridis")
    plt.title("Class Distribution")
    plt.show()

    # Check for missing values
    missing_values = data.isnull().sum()
    print("\nMissing Values:")
    print(missing_values[missing_values > 0])
    
    # Correlation Matrix for numerical features (if any)
    numerical_features = data.select_dtypes(include=['float64', 'int64']).columns
    if len(numerical_features) > 1:
        plt.figure(figsize=(12, 8))
        correlation_matrix = data[numerical_features].corr()
        sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
        plt.title("Correlation Matrix")
        plt.show()
    else:
        print("\nNot enough numerical features for correlation matrix.")

    print("\nEDA Complete.")


In [25]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns

# Load data from ndjson file
def load_ndjson(filepath: str) -> pd.DataFrame:
    """
    Load ndjson data and return as a pandas DataFrame.
    
    Args:
        filepath (str): Path to the ndjson file.
        
    Returns:
        pd.DataFrame: Data loaded into a pandas DataFrame.
    """
    with open(filepath, 'r') as file:
        data = [json.loads(line) for line in file]
    return pd.DataFrame(data)

# Define file path and load the data
filepath = r'C:\Users\eKasi_SWT_COM00862\Desktop\Health_Care_Project\Export v2 project - Health_Care_Interaction - 10_30_2024.ndjson'
data = load_ndjson(filepath)

# Extract true and predicted labels if available (customize based on your data structure)
true_labels = data['true_label'] if 'true_label' in data.columns else None
predicted_labels = data['predicted_label'] if 'predicted_label' in data.columns else None
predicted_probabilities = data['predicted_probabilities'] if 'predicted_probabilities' in data.columns else None

# Check if true_labels and predicted_labels are present for evaluation
if true_labels is not None and predicted_labels is not None:
    def evaluate_model_performance(true_labels: pd.Series, predicted_labels: pd.Series, predicted_probabilities: pd.Series = None):
        """
        Calculate and display key performance metrics of the model.
        
        Args:
            true_labels (pd.Series): Actual labels.
            predicted_labels (pd.Series): Predicted labels.
            predicted_probabilities (pd.Series, optional): Probability predictions for log loss.
        """
        # Calculate accuracy, precision, recall, F1, and log loss if probabilities available
        accuracy = accuracy_score(true_labels, predicted_labels)
        precision = precision_score(true_labels, predicted_labels, average='weighted')
        recall = recall_score(true_labels, predicted_labels, average='weighted')
        f1 = f1_score(true_labels, predicted_labels, average='weighted')
        
        print(f"\nModel Performance Metrics:\n")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        
        if predicted_probabilities is not None:
            logloss = log_loss(true_labels, predicted_probabilities)
            print(f"Log Loss: {logloss:.4f}")
    
    # Run model performance evaluation
    evaluate_model_performance(true_labels, predicted_labels, predicted_probabilities)

# EDA tailored for healthcare annotations
def perform_eda(data: pd.DataFrame):
    """
    Perform Exploratory Data Analysis (EDA) on healthcare annotation data.
    
    Args:
        data (pd.DataFrame): DataFrame containing annotations.
    """
    print("\n--- Exploratory Data Analysis ---")
    
    # Display basic info and summary statistics
    print("\nDataset Info:")
    print(data.info())
    
    print("\nDescriptive Statistics:")
    print(data.describe(include='all'))
    
    # Plot class distribution for annotations
    if 'label' in data.columns:
        plt.figure(figsize=(10, 6))
        sns.countplot(data=data, x='label', palette="viridis")
        plt.title("Class Distribution for Annotations")
        plt.show()
    
    # Check for missing values
    missing_values = data.isnull().sum()
    print("\nMissing Values:")
    print(missing_values[missing_values > 0])
    
    # Analyze entities, if present
    if 'entities' in data.columns:
        entity_counts = data['entities'].apply(lambda x: len(x) if isinstance(x, list) else 0)
        print("\nEntity Counts:")
        print(entity_counts.describe())
        plt.figure(figsize=(10, 6))
        sns.histplot(entity_counts, bins=20, kde=True)
        plt.title("Distribution of Entity Counts in Annotations")
        plt.show()

# Run EDA on the loaded data
perform_eda(data)


--- Exploratory Data Analysis ---

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   data_row          1 non-null      object
 1   media_attributes  1 non-null      object
 2   attachments       1 non-null      object
 3   metadata_fields   1 non-null      object
 4   projects          1 non-null      object
dtypes: object(5)
memory usage: 172.0+ bytes
None

Descriptive Statistics:
                                                 data_row  \
count                                                   1   
unique                                                  1   
top     {'id': 'cm2uh7qqp01860777dixjhfpx', 'external_...   
freq                                                    1   

                                         media_attributes attachments  \
count                                                   1           1   
u